In [1]:
import cPickle
import sys
sys.path.insert(0, '../../preprocess')
import vectorizer
beer_data = cPickle.load(open('../../beer_data/beer_vec_ds_df10.p', 'rb'))

Using TensorFlow backend.


[]


In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split
import numpy as np

ds = pd.read_csv('../../beer_data/beer_ds.csv')

from ast import literal_eval as make_tuple
ds['bits'] = ds['bits'].map(lambda s : make_tuple(s))

aspect_columns = sorted(['review/appearance', 'review/taste', 'review/aroma', 'review/palate'])
aspects = [str(i) for i in range(4)]

train_idxs, val_idxs = train_test_split(ds.index, stratify=ds['bits'], train_size=0.9, random_state=1337)

/home/sarthak/anaconda2/lib/python2.7/site-packages/sklearn/model_selection/_split.py:2026: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)


In [3]:
idxs = val_idxs
H = {}
labs = {}
for i in range(4) :
    H[str(i)] = np.zeros((len(idxs), len(idxs)))
    a0 = set(ds[ds[aspect_columns[i]] == 0].index) & set(idxs)
    a1 = set(ds[ds[aspect_columns[i]] == 1].index) & set(idxs)
    a0 = map(lambda s : list(idxs).index(s), a0)
    a1 = map(lambda s : list(idxs).index(s), a1)
#     H[str(i)][np.ix_(a0, a0)] = 1
#     H[str(i)][np.ix_(a1, a1)] = 1
    
    labs[str(i)] = np.array([0] * len(idxs))
    labs[str(i)][a1] = 1
    for j in a0 :
        H[str(i)][j, a0] = 1
    for j in a1 :
        H[str(i)][j, a1] = 1

    H[str(i)][np.arange(len(idxs)), np.arange(len(idxs))] = 0

In [4]:
from NVDMModel import log_softmax, cross_ent_loss, sample_norm

In [5]:
from keras.models import load_model
model = load_model('../store/weights/beer/Gated2CNNModelFriDec1517:16:522017dssm/loss.h5', 
                   custom_objects={'contrastive_loss' : lambda a,b:b})
#model = load_model('../store/weights/beer/CountModelWedDec620:08:322017coutnfinal/loss.h5',
#                   custom_objects={'contrastive_loss' : lambda a,b:b})
# model = load_model('../store/weights/beer/NVDMModelThuDec1418:20:122017nvdm19K/loss.h5',
#                   custom_objects={'contrastive_loss' : lambda a,b:b, 'log_softmax' : log_softmax,
#                                  'cross_ent_loss' : cross_ent_loss, 'sample_norm' : sample_norm,
#                                  '<lambda>' : lambda a,b : a})

In [6]:
import keras.backend as K
model_c1 = {}
for aspect in aspects :
    model_c1[aspect] = K.function(
                model.get_layer('pred_'+aspect).get_layer('pool_'+aspect).inputs + [K.learning_phase()], 
                model.get_layer('pred_'+aspect).get_layer('pool_'+aspect).outputs)

In [7]:
X_s = beer_data.X[val_idxs]
# X_tf = np.zeros((X_s.shape[0], beer_data.vocab_size))
# for i in range(len(X_s)) :
#     X_tf[i, X_s[i, :]] = 1.

# X_tf = X_tf[:, 2:]
# X_s = X_tf

In [8]:
embeds = {}
vecs = {k:[] for k in aspects}
i, bs = 0, 100
while i*bs < len(val_idxs):
    for aspect in aspects :
        result = model_c1[aspect]([X_s[i*bs:(i+1)*bs], 0])[0]
        vecs[aspect].append(result)
    i += 1

for aspect in aspects :
    embeds[aspect] = np.concatenate(vecs[aspect], axis=0)


In [10]:
#Gated CNN
from sklearn.preprocessing import normalize
from sklearn.metrics import roc_auc_score
for aspect in aspects :
    result = embeds[aspect]
    result = normalize(result, 'l2')
    scores = np.dot(result, result.T)
    scores[np.arange(len(val_idxs)), np.arange(len(val_idxs))] = -1000
    aucs = [0] * len(val_idxs)
    for aspect_j in aspects :
        for i in range(len(val_idxs)) :
            aucs[i] = roc_auc_score(H[aspect_j][i], scores[i])
        print aspect, aspect_j, np.mean(aucs)

0 0 0.911341239633
0 1 0.890595569948
0 2 0.885549865184
0 3 0.87598861092
1 0 0.899287180595
1 1 0.92787190655
1 2 0.914544674227
1 3 0.918786200896
2 0 0.900578244727
2 1 0.923714328519
2 2 0.928339228646
2 3 0.925721081735
3 0 0.898453918156
3 1 0.945021579464
3 2 0.945897428391
3 3 0.957836092953


In [10]:
#Count 
from sklearn.preprocessing import normalize
from sklearn.metrics import roc_auc_score
for aspect in aspects :
    result = embeds[aspect]
    result = normalize(result, 'l2')
    scores = np.dot(result, result.T)
    scores[np.arange(len(val_idxs)), np.arange(len(val_idxs))] = -1000
    aucs = [0] * len(val_idxs)
    for aspect_j in aspects :
        for i in range(len(val_idxs)) :
            aucs[i] = roc_auc_score(H[aspect_j][i], scores[i])
        print aspect, aspect_j, np.mean(aucs)

0 0 0.854554973644
0 1 0.820338060642
0 2 0.81058697348
0 3 0.797889746336
1 0 0.870872248819
1 1 0.894325436749
1 2 0.880327258009
1 3 0.880428469123
2 0 0.87033988649
2 1 0.890957378249
2 2 0.896394404544
2 3 0.890723541386
3 0 0.860553813433
3 1 0.907358169216
3 2 0.911861669863
3 3 0.923492443559


In [16]:
#NVDM
from sklearn.preprocessing import normalize
from sklearn.metrics import roc_auc_score
for aspect in aspects :
    result = embeds[aspect]
    result = normalize(result, 'l2')
    scores = np.dot(result, result.T)
    scores[np.arange(len(val_idxs)), np.arange(len(val_idxs))] = -1000
    aucs = [0] * len(val_idxs)
    for aspect_j in aspects :
        for i in range(len(val_idxs)) :
            aucs[i] = roc_auc_score(H[aspect_j][i], scores[i])
        print aspect, aspect_j, np.mean(aucs)

0 0 0.905304261335
0 1 0.877807419298
0 2 0.872206131665
0 3 0.860663657542
1 0 0.889331607361
1 1 0.917559396232
1 2 0.904842145736
1 3 0.905617773251
2 0 0.89315265191
2 1 0.917335669635
2 2 0.922035288076
2 3 0.918620585403
3 0 0.893972233173
3 1 0.940251184915
3 2 0.940928614247
3 3 0.951425403999


In [17]:
#Gated CNN
from sklearn.preprocessing import normalize
from sklearn.metrics import roc_auc_score
for aspect in aspects :
    result = embeds[aspect]
    result = normalize(result, 'l2')
    scores = np.dot(result, result.T)
    scores[np.arange(len(val_idxs)), np.arange(len(val_idxs))] = -1000
    
    for aspect_j in aspects :
        aucs_1, aucs_2 = [], []
        diff = np.where(labs[aspect] != labs[aspect_j])[0]
        for i, j in enumerate(diff) :
            aucs_1.append(roc_auc_score(H[aspect_j][j], scores[j]))
            aucs_2.append(roc_auc_score(H[aspect][j], scores[j]))

        print aspect, aspect_j, np.mean(aucs_1), np.mean(aucs_2)

0 0 nan nan


/home/sarthak/anaconda2/lib/python2.7/site-packages/numpy/core/fromnumeric.py:2909: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/home/sarthak/anaconda2/lib/python2.7/site-packages/numpy/core/_methods.py:80: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


0 1 0.418999945239 0.599924764743
0 2 0.398396384437 0.62942794504
0 3 0.38229051475 0.653758284764
1 0 0.328074756139 0.685847960222
1 1 nan nan
1 2 0.409582002789 0.593621923611
1 3 0.406335125999 0.604767116258
2 0 0.322878987074 0.695372910699
2 1 0.456705868057 0.544314099743
2 2 nan nan
2 3 0.485365467409 0.51949291894
3 0 0.231934426281 0.795875915478
3 1 0.346276551121 0.663425388414
3 2 0.334309001271 0.670527546249
3 3 nan nan


In [10]:
#Gated CNN
from sklearn.preprocessing import normalize
from sklearn.metrics import roc_auc_score
for aspect in aspects :
    result = embeds[aspect]
    result = normalize(result, 'l2')
    scores = np.dot(result, result.T)
    scores[np.arange(len(val_idxs)), np.arange(len(val_idxs))] = -1000
    aucs = [0] * len(val_idxs)
    for aspect_j in aspects :
        for i in range(len(val_idxs)) :
            aucs[i] = roc_auc_score(H[aspect_j][i], scores[i])
        print aspect, aspect_j, np.mean(aucs)

0 0 0.914870707499
0 1 0.883242027296
0 2 0.878992594595
0 3 0.86796036685
1 0 0.894897050529
1 1 0.929701394345
1 2 0.911345285882
1 3 0.9134474411
2 0 0.898631977632
2 1 0.920803103162
2 2 0.930926076498
2 3 0.926525263047
3 0 0.881059784896
3 1 0.934046852142
3 2 0.937719486865
3 3 0.952891638474


In [9]:
#Gated CNN
from sklearn.preprocessing import normalize
from sklearn.metrics import roc_auc_score
for aspect in aspects :
    result = embeds[aspect]
    result = normalize(result, 'l2')
    scores = np.dot(result, result.T)
    scores[np.arange(len(val_idxs)), np.arange(len(val_idxs))] = -1000
    aucs = [0] * len(val_idxs)
    for aspect_j in [aspect] :
        for i in range(len(val_idxs)) :
            aucs[i] = roc_auc_score(H[aspect_j][i], scores[i])
        print aspect, aspect_j, np.mean(aucs)

0 0 0.874847592559
1 1 0.89250408592
2 2 0.898233920927
3 3 0.922284604947
